In [ ]:
!pip install pyngrok

In [48]:
from pyngrok import ngrok
ngrok.kill()
NGROK_AUTH_TOKEN = "2Irwzpl15ewCiLLfkoTaiQN0YS3_mfT7Grs29VK5VZ1j14N4"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok.connect(8050)

<NgrokTunnel: "https://5f99-34-172-148-246.ngrok-free.app" -> "http://localhost:8050">

In [ ]:
!pip install dash
!pip install plotly

In [92]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Define your DataFrame df here or import it from an appropriate source
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# Layout Section of Dash
year_list = [i for i in range(1980, 2024, 1)]

# Create a Dash application and give it a meaningful title
app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1('Automobile Sales Statistics Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 24}),

    # outer division starts
    html.Div([
        # Add drop-down menus to your dashboard with appropriate titles and options
        # Dropdown to select statistics
        html.Div([
            dcc.Dropdown(id='dropdown-statistics',
                         options=[
                             {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                             {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
                         ],
                         placeholder='Select a report type',
                         value='Select Statistics',
                         style={'width': '80%', 'font-size': '20px', 'padding': '3px', 'text-align-last': 'center'}
                         )
        ]),
        # Dropdown to select year
        html.Div([
            dcc.Dropdown(id='select-year',
                         options=[{'label': i, 'value': i} for i in year_list],
                         placeholder='Select Year',
                         style={'width': '80%', 'font-size': '20px', 'padding': '3px', 'text-align-last': 'center'}
                         )]),

        # Add a division for output display with appropriate id and classname property
        html.Div([
            html.Div(id='output-container', className='chart-grid', style={'display': 'flex'}),
        ])
    ])
])

# Creating Callbacks; Define the callback function to update the input container based on the selected statistics and the
# output container

@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics', component_property='value'))
def update_input_container(dropdown_statistics):
    if dropdown_statistics == 'Yearly Statistics':
        return False
    else:
        return True


# Callback for plotting
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='select-year', component_property='value'),
     Input(component_id='dropdown-statistics', component_property='value')])
def update_output_container(select_year, dropdown_statistics):
    if dropdown_statistics == 'Recession Period Statistics':
        # Filter the data for recession periods
        recession_data = df[df['Recession'] == 1]

        # Plot 1 Automobile sales fluctuate over Recession Period (year wise) using line chart
        # grouping data for plotting
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        # Plotting the line graph
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec,
                            x='Year',
                            y='Automobile_Sales',
                            title="Automobile sales fluctuate over Recession Period (year wise)"))
         #Plot 2 Calculate the average number of vehicles sold by vehicle type and represent as a Bar chart
        vehicle_avg_sales=recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 =dcc.Graph(
             figure=px.bar(vehicle_avg_sales,
                          x='Vehicle_Type',
                          y='Automobile_Sales',
                          title='Average Number of Vehicles Sold by Vehicle Type (Recession Period)'))
         # Plot 3 : Pie chart for total expenditure share by vehicle type during recessions
            # grouping data for plotting
        exp_rec= recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
                    figure=px.pie(exp_rec,
                          values='Advertising_Expenditure',
                          names='Vehicle_Type',
                          title="Total expenditure share by vehicle type during recessions"))

        # Plot 4 Develop a Bar chart for the effect of unemployment rate on vehicle type and sales
        R_chart4 = dcc.Graph(
                     figure=px.bar(recession_data,
                          x='unemployment_rate',
                          y='Automobile_Sales',
                          color='Vehicle_Type',  # Color by Vehicle_Type
                          barmode='group',  # Group bars by unemployment_rate
                          title="Effect of Unemployment Rate on Vehicle Type and Sales"))
        return [
            html.Div(className='chart-item', children=[
                html.Div(children=R_chart1),
                html.Div(children=R_chart2)
            ]),
            html.Div(className='chart-item', children=[
                html.Div(children=R_chart3),
                html.Div(children=R_chart4)
            ])
        ]


    elif select_year and dropdown_statistics == 'Yearly Statistics':
        yearly_data = df[df['Year'] == select_year]
        # Plot 1: Yearly Automobile sales using line chart for the whole period.

        yas = df.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(figure=px.line(yas, x='Year', y='Automobile_Sales', title='Total Yearly Automobiles sale'))

        # Plot 2 :Total Monthly Automobile sales using line chart.
        yas = df.groupby('Month')['Automobile_Sales'].mean().reset_index()
        Y_chart2 = dcc.Graph(figure=px.line(yas, x='Month', y='Automobile_Sales', title='Total Monthly Automobiles sale'))
        # Plot bar chart for average number of vehicles sold during the given year

        avr_vdata=yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph( figure=px.bar(avr_vdata,x='Vehicle_Type',y='Automobile_Sales',title='Average Vehicles Sold by Vehicle Type in the year {}'.format(select_year)))
       # Plot 4 Total Advertisement Expenditure for each vehicle using pie chart
        Y_chart4 = dcc.Graph(
                    figure=px.pie(yearly_data,
                          values='Advertising_Expenditure',
                          names='Vehicle_Type',
                          title="Total Advertisement Expenditure for each vehicle using pie chart"))
        return [
            html.Div(className='chart-item', children=[
                html.Div(children=Y_chart1),
                html.Div(children=Y_chart2)
            ]),
            html.Div(className='chart-item', children=[
                html.Div(children=Y_chart3),
                html.Div(children=Y_chart4)
            ])
        ]
    else:
        return None

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>